In [7]:
import dash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as  px
import pandas as pd
import numpy as np

In [2]:
education = pd.read_csv("states_all.csv").assign(
    expenditure_per_student = lambda x: x["TOTAL_EXPENDITURE"] / x["GRADES_ALL_G"],
    above_avg_math8 = lambda x: np.where(x["AVG_MATH_8_SCORE"] > 278.6,'Above Avg', 'Below Avg')
)

In [ ]:
# Dropdown example
app= Dash(__name__)

app.layout=html.Div([
    dcc.Dropdown(
        id="filter",
        options=education["STATE"].unique(),
        value=["CALIFORNIA"],
        multi=True
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output('graph','figure'),
              Input('filter','value'))

def  plot_rev_scatter(state):
    fig=px.line(
        (
            education.query("STATE in @state and 1992 < YEAR < 2016")
            .groupby(["YEAR","STATE"])
            .sum()
            .reset_index()
        ),
        x='YEAR',
        y='TOTAL_EXPENDITURE',
        color='STATE',
        title=f"Expenditure Over Time, Selected States",
    )
    return fig

if __name__=="__main__":
    app.run_server(debug=True)

In [ ]:
# Checklist example
app= dash.Dash(__name__)
app.layout=html.Div([
    dcc.Checklist(
        id="filter",
        #options=['CALIFORNIA','TEXAS','FLORIDA','OREGON'],
        options=[{'label':state.title(), 'value':state} for state in education["STATE"].unique()],
        value=['CALIFORNIA','TEXAS']
    ),
    dcc.Graph(id="graph")
])

@app.callback(Output('graph','figure'),
              Input('filter','value'))

def  plot_rev_scatter(state):
    fig=px.line(
        (
            education.query("STATE in @state and 1992 < YEAR < 2016")
            .groupby(["YEAR","STATE"])
            .sum()
            .reset_index()
        ),
        x='YEAR',
        y='TOTAL_EXPENDITURE',
        color='STATE',
        title=f"Expenditure Over Time, Selected States",
    )
    return fig

if __name__=="__main__":
    app.run_server(debug=True)

In [ ]:
# example for radio buttons
app=dash.Dash(__name__)

app.layout=html.Div(
    [dcc.RadioItems(
        id='aggfunc',
        options=['mean','sum','median'],
        value='mean'
    ),
    dcc.Graph(id='graph')]
)

@app.callback(Output('graph','figure'), Input('aggfunc','value'))

def plot_rev_scatter(metric):
    fig=px.bar(
        (education.query("STATE in ['CALIFORNIA','OREGON','WASHINGTON','IDAHO'] and 1992 < YEAR < 2014")
         .groupby(['STATE'])
         .agg({'TOTAL_EXPENDITURE':f"{metric}"})
         .reset_index()),
         x='STATE',
         y='TOTAL_EXPENDITURE',
         text_auto=True,
         title=f"Expenditure Over Time, Selected States"
    )
    return fig

if __name__=='__main__':
    app.run_server(debug=True)

In [ ]:
# example for sliders
app= dash.Dash(__name__)

app.layout=html.Div(
    [
        dcc.Slider(
            id='score_slider',
            min=education['AVG_MATH_8_SCORE'].min(),
            max=education['AVG_MATH_8_SCORE'].max(),
            step=10,
            value=education['AVG_MATH_8_SCORE'].mean()
        ),
        dcc.Graph(id='graph')
    ]
)

@app.callback(Output('graph','figure'),Input('score_slider','value'))

def plot_rev_scatter(score):
    df=(
        education.query("YEAR==2013")
        .assign(score_cutoff=lambda x: np.where(
            x["AVG_MATH_8_SCORE"]>score,
            'Above Threshold',
            'Below Threshold'
        ))
    )
    fig=px.scatter(
        df,
        x='AVG_MATH_8_SCORE',
        y='expenditure_per_student',
        color='score_cutoff',
        hover_name='STATE',
        title="Score and Expenditure vs Threshold"
    )
    return fig


if __name__=='__main__':
    app.run_server(debug=True)

In [ ]:
# example for range slider

app=dash.Dash(__name__)

app.layout=html.Div(
    [
        dcc.RangeSlider(
            id='score_slider',
            min=education.query('YEAR==2013')['AVG_MATH_8_SCORE'].min(),
            max=education.query('YEAR==2013')['AVG_MATH_8_SCORE'].max(),
            step=5,
            value=[270,280]
        ),
        dcc.Graph(id='graph')
    ]
)

@app.callback(Output('graph','figure'),Input('score_slider','value'))

def plot_rev_scatter(score):
    df=(
        education.query('YEAR==2013')
        .assign(score_cutoff=lambda x:np.where(
            x['AVG_MATH_8_SCORE'].between(score[0],score[1]),
            'In Range',
            'Out of Range'
        ))
    )
    fig=px.scatter(
        df,
        x='AVG_MATH_8_SCORE',
        y='expenditure_per_student',
        color='score_cutoff',
        hover_name='STATE',
        title='Score and expenditure vs Threshold'
    )
    return fig

if __name__=='__main__':
    app.run_server(debug=True)
    


In [ ]:
collisions = (
    pd.read_csv("NYC_Collisions.csv")
    .astype({"ACCIDENT_DATE": "datetime64[ns]"})
    .loc[:,["ACCIDENT_DATE", "BOROUGH", "COLLISION_ID"]]
    .groupby(["ACCIDENT_DATE", "BOROUGH"], as_index=False)
    .count()
    .rename(columns={
        "COLLISION_ID":"COLLISIONS", 
        "ACCIDENT_DATE":"DATE"})  
)

collisions.head()

In [ ]:
# Date picker demo
app=Dash(__name__)

app.layout=html.Div(
    [
        dcc.DatePickerSingle(
            id='date_picker',
            min_date_allowed=collisions['DATE'].min(),
            max_date_allowed=collisions['DATE'].max(),
            initial_visible_month=collisions['DATE'].max(),
            date=collisions['DATE'].max(),
            display_format='YYYY-MM-DD'
        ),
        dcc.Graph(id='graph')
    ]
)

@app.callback(Output('graph','figure'),Input('date_picker',"date"))

def plot_collisions_bar(date):
    fig=px.bar(
        collisions.loc[collisions['DATE'].eq(date)],
        x='COLLISIONS',
        y='BOROUGH',
        title=f"Traffic Accidents in NYC on {date[:10]}"
    )
    return fig

app.run_server(debug=True)


In [25]:
# Date picker range

app=Dash(__name__)

app.layout=html.Div(
    [
        dcc.DatePickerRange(
            id='date_picker',
            min_date_allowed=collisions['DATE'].min(),
            max_date_allowed=collisions['DATE'].max(),
            initial_visible_month=collisions['DATE'].max(),
            start_date=collisions['DATE'].min(),
            end_date=collisions['DATE'].max(),
            display_format='YYYY-MM-DD'
        ),
        dcc.Graph(id='graph')
    ]
)

@app.callback(Output('graph','figure'),[Input('date_picker',"start_date"),Input('date_picker','end_date')])

def plot_collisions_bar(start_date,end_date):
    fig=px.bar(
        (collisions.loc[collisions['DATE'].between(start_date,end_date)]
        .groupby('BOROUGH',as_index=False)
        .sum()),
        x='COLLISIONS',
        y='BOROUGH',
        title=f"Traffic Accidents in NYC in between {start_date[:10]} and {end_date[:10]}" 
    )
    return fig

app.run_server(debug=True)


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[25], line 26, in plot_collisions_bar(
    start_date='2021-01-01T00:00:00',
    end_date='2022-02-28T00:00:00'
)
     20 @app.callback(Output('graph','figure'),[Input('date_picker',"start_date"),Input('date_picker','end_date')])
     21 
     22 def plot_collisions_bar(start_date,end_date):
     23     fig=px.bar(
     24         (collisions.loc[collisions['DATE'].between(start_date,end_date)]
     25         .groupby('BOROUGH',as_index=False)
---> 26         .sum()),
        px = <module 'plotly.express' from '/home/akash/plotly_and_dash-main/myenv/lib/python3.12/site-packages/plotly/express/__init__.py'>
        start_date = '2021-01-01T00:00:00'
        end_date = '2022-02-28T00:00:00'
        collisions =            DATE        BOROUGH  COLLISIONS
0    2021-01-01          BRONX          28
1    2021-01-01       BROOKLYN     